In [1]:
!ln -s /Users/adel.schmucklermann/Desktop/library.py library.py

In [2]:
import pytorch_lightning as pl
import h5py
import os
#import pandas as pd
import numpy as np
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import library as lib
import pandas as pd
#from library import makedataset

In [3]:
pl.seed_everything(42, workers=True) 

Global seed set to 42


42

In [4]:
#make numpy file with (seq header, embedding)
esm2 = h5py.File("/Users/adel.schmucklermann/Desktop/FoPra/KLK/KLK_esm2.h5", 'r')
list(esm2.keys())
# List all groups
keys = list(esm2.keys())
#print(keys) 
# Get the data
embedding = []
headers = []
for key in keys:
    #data
    emb = esm2[key][:]
    embedding.append(emb)
    #headers
    key = key.replace("isoform=", "isoform_").replace("=", "").replace(":", "_")
    headers.append(key)

headers = np.array(headers)
embedding = np.array(embedding)

#split train test
headers_first, headers_test, embedding_first, embedding_test = train_test_split(headers, embedding, test_size=0.1, random_state=42)
#split train validation
headers_train, headers_validate, embedding_train, embedding_validate = train_test_split(headers_first, embedding_first, test_size =0.2, random_state=42)
print("total:", headers.shape)
print("training+validation:", headers_first.shape)
print("test:", headers_test.shape)
print("training:", headers_train.shape)
print("validation:", headers_validate.shape)
print(len(headers_train), len(embedding_train))
print(len(headers_validate), len(embedding_validate))

#training_df = pd.DataFrame({'header': headers_train, 'embedding' : embedding_train})
#validate_df = pd.DataFrame({'header': headers_validate, 'embedding' : embedding_validate})

total: (437,)
training+validation: (393,)
test: (44,)
training: (314,)
validation: (79,)
314 314
79 79


In [5]:
#dataloaders
train_dataset = lib.makedataset(headers_train, embedding_train)
val_dataset = lib.makedataset(headers_validate, embedding_validate)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
    

In [6]:
#if prott5 used: dtype=float16 -> error with .linear()

#encoder_layers, latent_dim, decoder_layers
dim = 2560
a = int(dim * 1/16)
b = int(dim * 1/8)
c = int(dim * 1/2)
encoder_layers = [dim, c, b, a]
latent_dim = a
decoder_layers = [a, b, c, dim]

# Initialize the VAE model
vae = lib.VariationalAutoencoder(encoder_layers, latent_dim, decoder_layers)

trainer = pl.Trainer(accelerator="auto", max_epochs=50, check_val_every_n_epoch=1)
#trainer = pl.Trainer(accelerator="gpu", devices = 1, max_epochs=500, check_val_every_n_epoch=1)

# Train the VAE model
trainer.fit(vae, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 3.7 M 
1 | latent  | Sequential | 26.1 K
2 | fc21    | Linear     | 25.8 K
3 | fc22    | Linear     | 25.8 K
4 | fc3     | Sequential | 26.1 K
5 | fc4     | Sequential | 26.1 K
6 | relu    | ReLU       | 0     
7 | decoder | Sequential | 3.8 M 
---------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.491    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/adel.schmucklermann/miniconda3/envs/chumby/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/adel.schmucklermann/miniconda3/envs/chumby/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 64. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/adel.schmucklermann/miniconda3/envs/chumby/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consi

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [7]:
#Validate
trainer.validate(vae, val_dataloader)

Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      validation_loss      │    2.2888402938842773     │
└───────────────────────────┴───────────────────────────┘

[{'validation_loss': 2.2888402938842773}]

In [8]:
# Predict

test_dataset = lib.makepredictset(embedding_test) #headers_test, 
test_dataloader = DataLoader(test_dataset, shuffle=False, num_workers=0, drop_last=True)



#trainer = pl.Trainer()
prediction = trainer.predict(vae,test_dataloader)
print(len(prediction))
print(len(headers_test))

/Users/adel.schmucklermann/miniconda3/envs/chumby/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 4it [00:00, ?it/s]

44
44


In [18]:
# Neighbor_joining and create tree

embeddings_pred = []
for i in prediction:
    print("i: ",i)
    embeddings_pred.insert(i.numpy())
#prediction_np = prediction.numpy()
newick = lib.neighbor_joining(embeddings_pred,headers_test)

i:  (tensor([[ 0.4262,  1.2414, -0.3466,  ..., -0.2938, -1.7813,  1.2146]]), tensor([[-2.7607e-03,  1.4199e-02,  5.8328e-03,  7.7991e-03,  1.1201e-02,
         -1.1630e-02,  2.9885e-03, -9.7944e-04, -4.7585e-04,  5.3422e-03,
         -1.8171e-02, -3.9839e-03,  3.6423e-03, -6.7160e-03,  7.2468e-03,
          5.3745e-03, -5.4425e-03,  1.9820e-02, -5.4720e-03,  5.9198e-05,
          2.3505e-02,  3.6022e-03, -1.3116e-02, -1.1477e-02,  2.2846e-03,
         -4.1002e-03,  8.9607e-03,  4.8080e-03,  2.7430e-03,  8.6384e-03,
          1.3229e-02, -1.5578e-02,  2.9508e-03, -8.7234e-03,  1.4492e-02,
         -1.0132e-02, -1.4136e-02, -5.9681e-03,  2.5989e-02,  5.8010e-03,
         -1.8258e-03,  1.0668e-02,  2.0026e-03,  4.1167e-03,  7.1167e-04,
         -1.1499e-02,  2.3102e-05, -9.0944e-03, -1.6744e-03,  1.9796e-02,
         -2.0305e-03,  2.1961e-04,  1.1929e-02, -4.7323e-03, -8.3104e-03,
          5.6853e-04, -7.1471e-03, -1.3118e-03, -3.5286e-03,  1.2469e-02,
         -3.0232e-03,  4.2964e-03, 

AttributeError: 'tuple' object has no attribute 'numpy'

In [ ]:
#Plot
# read csv file
df = pd.read_csv('lightning_logs/version_54/metrics.csv')

# plot
plt.plot('epoch', 'loss', data=df)
plt.plot('epoch', 'reconstruction_loss', data=df)
plt.plot('epoch', 'kl_loss', data=df)
plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['loss', 'reconstruction loss', 'kl loss'], loc='upper right')
plt.show()